I decided to make this notebook for my learning reference after I learned that image NN models often use encoders and decoders in order to minimize image sizes thus reducing better acquiring data for the purpose of model training.

Keeping in mind my goal is to see if compression is possible with ML I think messing around more with this to learn about this mechanisim is a good idea.

In [3]:
#Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt